## Overview: Waymo Open Dataset -- Perception Object Assets

Modeling the 3D world from sensor data for simulation is a scalable way of developing testing and validation environments for robotic learning problems such as autonomous driving. We provide a large-scale object-centric asset dataset containing images and lidar observations of two major object categories (`VEHICLE` and `PEDESTRIAN`) from the released Perception data (v2.0.0). We hope this data will enable and advance research on 3D point cloud reconstruction and completion, object NeRF reconstruction, and generative object assets to address the real-world driving challenges with occlusions, lighting-variations, and long-tail distributions.

Please familiarize yourself with the [Perception data v2 format and tutorial](https://github.com/waymo-research/waymo-open-dataset) then proceed.

##Get Prepared

As a prerequisite, please download the object asset components and the relevant [`LiDARBoxComponent`](src/waymo_open_dataset/v2/perception/box.py) for this tutorial.

The object asset components include:
* `ObjectAssetLiDARSensorComponent`: LiDAR points within each bounding box;
* `ObjectAssetCameraSensorComponent`: Camera image patches, cropping config, and projected LiDAR points;
* `ObjectAssetRayComponent`: Camera rays transformed to the bounding box coordinate frame;
* `ObjectAssetAutoLabelComponent`: Dense auto-labels on the camera image patches;
* `ObjectAssetRefinedPoseComponent`: Optional refined bounding box;
* `ObjectAssetRayCompressedComponent`: A compressed copy of the corresponding `ObjectAssetRayComponent`.

In [0]:
#@title Access GCP Bucket

from google.colab import auth
import gcsfs
import google.auth
auth.authenticate_user()

credentials, project_id = google.auth.default()
fs = gcsfs.GCSFileSystem(project=project_id, token=credentials)

In [0]:
#@title Load data from GCP Bucket

from typing import Optional, Any, Mapping, Tuple
import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)

import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
import plotly  # Used by visu3d
import tensorflow as tf
import visu3d

from waymo_open_dataset import v2
from waymo_open_dataset.utils import camera_segmentation_utils
object_asset_utils = v2.object_asset_utils

#@markdown Specify data split
data_split = 'validation' #@param ["validation", "training"]
# Path to the directory with all components of perception dataset.
dataset_dir = dataset_dir = f'waymo_open_dataset_v_2_0_0/{data_split}'
# Path to the directory with all components of the perception object asset dataset.
asset_dir = dataset_dir

context_name = '4195774665746097799_7300_960_7320_960'

def read(dataset_dir: str, tag: str) -> dd.DataFrame:
  """Creates a Dask DataFrame for the component specified by its tag."""
  paths = fs.glob(f'{dataset_dir}/{tag}/{context_name}.parquet')
  return dd.read_parquet(paths, filesystem=fs)


def read_codec(dataset_dir: str, tag: str) -> Any:
  """Reads the codec file for 3D ray decompression."""
  codec_path = fs.glob(f'{dataset_dir}/{tag}/codec_config.json')[0]
  with tf.io.gfile.GFile(codec_path, 'r') as f:
    codec_config = v2.ObjectAssetRayCodecConfig.from_json(f.read())
  return v2.ObjectAssetRayCodec(codec_config)


In [0]:
#@title List object assets from run

#@markdown Select object category
asset_type = 'veh' #@param ["veh", "ped"]

asset_df = read(asset_dir, f'{asset_type}_asset_camera_sensor')
grouped_asset_df = asset_df.groupby(['key.laser_object_id'])
unique_object_df = grouped_asset_df['key.laser_object_id'].unique().compute()

num_assets = unique_object_df.shape[0]
print(f'Available {num_assets} unique objects:')
for i in range(num_assets):
  sel_asset_df = grouped_asset_df.get_group(
      unique_object_df.iat[i][0])
  num_frames = len(sel_asset_df)
  _, r = next(sel_asset_df.iterrows())
  camera_sensor_component = v2.ObjectAssetCameraSensorComponent.from_dict(r)
  print(
    'laser_object_id: ', camera_sensor_component.key.laser_object_id,
    ' num_of_frames: ', num_frames)
  plt.imshow(camera_sensor_component.rgb_image_numpy)
  plt.show()

## Load and visualize camera fields

In [0]:
#@title Setup visualization functions
def apply_color_mask(image: np.ndarray,
                     mask: np.ndarray,
                     color: Tuple[int, int, int],
                     alpha: float = 0.5) -> np.ndarray:
  """Applies the given mask to the image."""
  color = np.array(color)[np.newaxis, :]
  bg = image * (1 - alpha) + alpha * color
  output = np.where(mask, bg, image).astype(np.uint8)
  return output


def grid_imshow(h: int, w: int, images: Any) -> None:
  """Displays images in a grid."""
  fig, axes = plt.subplots(h, w)
  fig.set_size_inches(20, 10)
  fig.tight_layout()
  for i, image in enumerate(images):
    ax = axes[i] if len(images) > 0 else axes
    ax.imshow(image)
    ax.axis('off')

In [0]:
#@title Select object asset
asset_camera_sensor_df = read(asset_dir, f'{asset_type}_asset_camera_sensor')
asset_ray_df = read(asset_dir, f'{asset_type}_asset_ray')
asset_auto_label_df = read(asset_dir, f'{asset_type}_asset_auto_label')
# Load additional LiDAR box dimensions to obtain the ray-box intersection
laser_box_df = read(dataset_dir, 'lidar_box')

asset_df = v2.merge(asset_camera_sensor_df, asset_ray_df)
asset_df = v2.merge(asset_df, asset_auto_label_df)
asset_df = v2.merge(asset_df, laser_box_df)

grouped_asset_df = asset_df.groupby(['key.laser_object_id'])
unique_object_df = grouped_asset_df['key.laser_object_id'].unique().compute()

#@markdown Specify the asset index for loading
sel_index = 0 #@param {type: "integer"}
sel_asset_df = grouped_asset_df.get_group(
    unique_object_df.iat[sel_index][0]).reset_index()

In [0]:
#@title Visualize camera patches

def _compute_ray_mask(ray_origin: np.ndarray,
                      ray_direction: np.ndarray,
                      box_size: np.ndarray) -> np.ndarray:
  im_height, im_width = ray_origin.shape[:2]
  ray_mask, _, _ = object_asset_utils.get_ray_box_intersects(
      ray_component.ray_origin.numpy.reshape(-1, 3),
      ray_component.ray_direction.numpy.reshape(-1, 3),
      lidar_box_component.box.size.numpy,
  )
  return ray_mask.reshape(im_height, im_width, -1)

# Example how to access data fields.
print(f'Available {sel_asset_df.shape[0].compute()} rows:')
for i, (_, r) in enumerate(sel_asset_df.iterrows()):
  # Create component dataclasses for the raw data
  lidar_box_component = v2.LiDARBoxComponent.from_dict(r)
  camera_sensor_component = v2.ObjectAssetCameraSensorComponent.from_dict(r)
  auto_label_component = v2.ObjectAssetAutoLabelComponent.from_dict(r)
  ray_component = v2.ObjectAssetRayComponent.from_dict(r)

  ray_mask = _compute_ray_mask(
      ray_component.ray_origin.numpy,
      ray_component.ray_direction.numpy,
      lidar_box_component.box.size.numpy)

  panoptic_image = camera_segmentation_utils.panoptic_label_to_rgb(
      semantic_label=auto_label_component.semantic_mask_numpy,
      instance_label=auto_label_component.instance_mask_numpy)

  print(
      'context_name: ', lidar_box_component.key.segment_context_name,
      ' ts: ', lidar_box_component.key.frame_timestamp_micros,
      ' laser_object_id: ', lidar_box_component.key.laser_object_id)
  grid_imshow(1, 5, [
      camera_sensor_component.rgb_image_numpy,
      apply_color_mask(
          camera_sensor_component.rgb_image_numpy,
          camera_sensor_component.proj_points_mask_numpy,
          color=(0, 255, 0),
          alpha=0.7),
      apply_color_mask(
          camera_sensor_component.rgb_image_numpy,
          ray_mask,
          color=(0, 255, 255),
          alpha=0.3),
      apply_color_mask(
          camera_sensor_component.rgb_image_numpy,
          auto_label_component.object_mask_numpy ,
          color=(255, 255, 0),
          alpha=0.3),
          panoptic_image])
  plt.show()
  if i > 2:
    print('...')
    break


We unproject each camera pixels to 3D and visualize the corresponding rays.
Note that these images are captured with five cameras mounted on the car. Please
refer to the Waymo Open Dataset -- Perception ([arxiv](https://arxiv.org/abs/1912.04838), [website](https://waymo.com/open/)) for details about camera sensor configurations.
- `FRONT`
- `FRONT_LEFT`
- `FRONT_RIGHT`
- `SIDE_LEFT`
- `SIDE_RIGHT`

### Visualization: object-centric coordinate

Assuming objects in the **`VEHICLE`** category have rigid shape across all frames, we visualize any single frame and aggregated frames in the object-centric coordinate.

In [0]:
#@title Unproject pixels in 3D and camera rays (one frame)

for i, (_, r) in enumerate(sel_asset_df.iterrows()):
  lidar_box_component = v2.LiDARBoxComponent.from_dict(r)
  camera_sensor_component = v2.ObjectAssetCameraSensorComponent.from_dict(r)
  auto_label_component = v2.ObjectAssetAutoLabelComponent.from_dict(r)
  ray_component = v2.ObjectAssetRayComponent.from_dict(r)

  ray_dists = camera_sensor_component.proj_points_dist.numpy.reshape(-1, 1)
  ray_origins = ray_component.ray_origin.numpy.reshape(-1, 3)
  ray_directions = ray_component.ray_direction.numpy.reshape(-1, 3)
  # Unproject pixels to 3D.
  ray_points = ray_dists * ray_directions + ray_origins
  ray_colors = camera_sensor_component.rgb_image_numpy.reshape(-1, 3)

  ray_masks = camera_sensor_component.proj_points_mask_numpy.reshape(-1)
  ray_masks *= auto_label_component.object_mask_numpy.reshape(-1)

  if ray_masks.sum() > 0:
    ray_ind = np.where(ray_masks > 0)[0]
    v3d_points = visu3d.Point3d(
        p=ray_points[ray_ind, :], rgb=ray_colors[ray_ind, :])
    v3d_rays = visu3d.Ray(
        pos=ray_origins[ray_ind, :], dir=ray_directions[ray_ind, :])
    visu3d.make_fig([v3d_points, v3d_rays])
  break



In [0]:
#@title Unproject pixels in 3D and camera rays (aggregated frames)

v3d_rays = []
v3d_points = []

for i, (_, r) in enumerate(sel_asset_df.iterrows()):
  lidar_box_component = v2.LiDARBoxComponent.from_dict(r)
  camera_sensor_component = v2.ObjectAssetCameraSensorComponent.from_dict(r)
  auto_label_component = v2.ObjectAssetAutoLabelComponent.from_dict(r)
  ray_component = v2.ObjectAssetRayComponent.from_dict(r)

  ray_dists = camera_sensor_component.proj_points_dist.numpy.reshape(-1, 1)
  ray_origins = ray_component.ray_origin.numpy.reshape(-1, 3)
  ray_directions = ray_component.ray_direction.numpy.reshape(-1, 3)
  # Unproject pixels to 3D.
  ray_points = ray_dists * ray_directions + ray_origins
  ray_colors = camera_sensor_component.rgb_image_numpy.reshape(-1, 3)

  ray_masks = camera_sensor_component.proj_points_mask_numpy.reshape(-1)
  ray_masks *= auto_label_component.object_mask_numpy.reshape(-1)

  if ray_masks.sum() > 0:
    ray_ind = np.where(ray_masks > 0)[0]
    v3d_points.append(
        visu3d.Point3d(
            p=ray_points[ray_ind, :], rgb=ray_colors[ray_ind, :]))
    v3d_rays.append(
        visu3d.Ray(
            pos=ray_origins[ray_ind, :], dir=ray_directions[ray_ind, :]))

visu3d.make_fig(v3d_points + v3d_rays)

## Load and visualize point clouds

In [0]:
#@title Visualize lidar points
asset_df = read(asset_dir, f'{asset_type}_asset_lidar_sensor')

grouped_asset_df = asset_df.groupby(['key.laser_object_id'])
unique_object_df = grouped_asset_df[
    'key.laser_object_id'].unique().compute()

sel_asset_df = grouped_asset_df.get_group(
    unique_object_df.iat[sel_index][0]).reset_index()

all_points_xyz = []
for i, (_, r) in enumerate(sel_asset_df.iterrows()):
  # Create component dataclasses for the raw data
  lidar_sensor_component = v2.ObjectAssetLiDARSensorComponent.from_dict(r)
  print(
      f'context_name: {lidar_sensor_component.key.segment_context_name}',
      f' ts: {lidar_sensor_component.key.frame_timestamp_micros}',
      f' laser_object_id: {lidar_sensor_component.key.laser_object_id}')

  all_points_xyz.append(lidar_sensor_component.points_xyz.numpy)
  if i > 9:
    break

v3d_point_cloud = visu3d.Point3d(
    p=np.concatenate(all_points_xyz, axis=0),
    rgb=(255, 0, 0),
)
v3d_point_cloud.fig

Human labled 3D laser bounding boxes in the Waymo Open Dataset -- Perception
are designed for the object detection task. Given a sequence of 3D laser bounding boxes, we further refine the boxes through 3D point cloud registration and provide the refined box pose (as 4x4 rigid transformation) per observation in the **`VEHICLE`** category.


In [0]:
#@title Visualize and compare lidar points (with refined box pose)

asset_lidar_sensor_df = read(asset_dir, 'veh_asset_lidar_sensor')
# Note: Registered Box Pose is only defined for vehicle objects.
asset_refined_pose_df = read(asset_dir, 'veh_asset_refined_pose')

# Load additional LiDAR box dimensions to obtain the ray-box intersection
laser_box_df = read(dataset_dir, 'lidar_box')

asset_df = v2.merge(asset_lidar_sensor_df, asset_refined_pose_df)
asset_df = v2.merge(asset_df, laser_box_df)

grouped_asset_df = asset_df.groupby(['key.laser_object_id'])
unique_object_df = grouped_asset_df[
    'key.laser_object_id'].unique().compute()

sel_asset_df = grouped_asset_df.get_group(
    unique_object_df.iat[sel_index][0]).reset_index()

all_points_xyz = []
for i, (_, r) in enumerate(sel_asset_df.iterrows()):
  # Create component dataclasses for the raw data
  lidar_sensor_component = v2.ObjectAssetLiDARSensorComponent.from_dict(r)
  refined_pose_component = v2.ObjectAssetRefinedPoseComponent.from_dict(r)
  lidar_box_component = v2.LiDARBoxComponent.from_dict(r)
  print(
      f'context_name: {lidar_sensor_component.key.segment_context_name}',
      f' ts: {lidar_sensor_component.key.frame_timestamp_micros}',
      f' laser_object_id: {lidar_sensor_component.key.laser_object_id}')
  points_xyz = lidar_sensor_component.points_xyz.numpy
  box_3d = lidar_box_component.box.numpy(dtype=np.float64)
  refined_box_from_vehicle = np.asarray(
      refined_pose_component.box_from_vehicle.transform,
      dtype=np.float64).reshape(4, 4)
  points_xyz_tfm = object_asset_utils.transform_points_to_box_coord_reverse(
      points_xyz, box_3d)
  points_xyz_refined = object_asset_utils.transform_points_to_frame(
      points_xyz_tfm, refined_box_from_vehicle
  )
  all_points_xyz.append(points_xyz_refined)
  if i > 9:
    break

v3d_point_cloud_refined = visu3d.Point3d(
    p=np.concatenate(all_points_xyz, axis=0),
    rgb=(0, 0, 255),
)
visu3d.make_fig([v3d_point_cloud_refined, v3d_point_cloud])

## Access compressed 3D rays

Besides the accurate 3D rays for every camera pixel, we provide a compressed version for the corresponding 3D rays. This results in a significant reduction
in dataset size, while introducing a neglible error (~1e-5 m) in ray origins and
directions.


In [0]:
#@title Setup ray compressed component

# The codec that is used to decompress the ray compressed component.
codec = read_codec(asset_dir, f'{asset_type}_asset_ray_compressed')

asset_ray_compressed_df = read(asset_dir, f'{asset_type}_asset_ray_compressed')
asset_camera_sensor_df = read(asset_dir, f'{asset_type}_asset_camera_sensor')

# Load additional LiDAR box dimensions to obtain the ray-box intersection
laser_box_df = read(dataset_dir, 'lidar_box')

asset_df = v2.merge(asset_camera_sensor_df, asset_ray_compressed_df)
asset_df = v2.merge(asset_df, laser_box_df)

grouped_asset_df = asset_df.groupby(['key.laser_object_id'])
unique_object_df = grouped_asset_df['key.laser_object_id'].unique().compute()

sel_asset_df = grouped_asset_df.get_group(
    unique_object_df.iat[sel_index][0]).reset_index()

In [0]:
#@title Unproject pixels in 3D and visualize compressed camera rays (one frame)

it = sel_asset_df.iterrows()

_, r = next(it)
ray_compressed_component = v2.ObjectAssetRayCompressedComponent.from_dict(r)
ray_component = codec.decode(ray_compressed_component)
camera_sensor_component = v2.ObjectAssetCameraSensorComponent.from_dict(r)

ray_dists = camera_sensor_component.proj_points_dist.numpy.reshape(-1, 1)
ray_origins = ray_component.ray_origin.numpy.reshape(-1, 3)
ray_directions = ray_component.ray_direction.numpy.reshape(-1, 3)

ray_points = ray_dists * ray_directions + ray_origins
ray_colors = camera_sensor_component.rgb_image_numpy.reshape(-1, 3)

ray_masks = camera_sensor_component.proj_points_mask_numpy.reshape(-1)

ray_ind = np.where(ray_masks > 0)[0]
v3d_points = visu3d.Point3d(
    p=ray_points[ray_ind, :], rgb=ray_colors[ray_ind, :])
v3d_rays = visu3d.Ray(
    pos=ray_origins[ray_ind, :], dir=ray_directions[ray_ind, :])
visu3d.make_fig([v3d_points, v3d_rays])